In [24]:
from pyspark.sql import SparkSession

In [25]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [26]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [27]:
rdd = sc.textFile("s3://msbx5420-2020/team-blanca-peak/yellow_tripdata_2019-01.csv")
rdd

s3://msbx5420-2020/team-blanca-peak/yellow_tripdata_2019-01.csv MapPartitionsRDD[13] at textFile at NativeMethodAccessorImpl.java:0

In [28]:
rdd.take(10)

['VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge',
 '1,1/1/2019 0:46,1/1/2019 0:53,1,1.5,1,N,151,239,1,7,0.5,0.5,1.65,0,0.3,9.95,',
 '1,1/1/2019 0:59,1/1/2019 1:18,1,2.6,1,N,239,246,1,14,0.5,0.5,1,0,0.3,16.3,',
 '2,12/21/2018 13:48,12/21/2018 13:52,3,0,1,N,236,236,1,4.5,0.5,0.5,0,0,0.3,5.8,',
 '2,11/28/2018 15:52,11/28/2018 15:55,5,0,1,N,193,193,2,3.5,0.5,0.5,0,0,0.3,7.55,',
 '2,11/28/2018 15:56,11/28/2018 15:58,5,0,2,N,193,193,2,52,0,0.5,0,0,0.3,55.55,',
 '2,11/28/2018 16:25,11/28/2018 16:28,5,0,1,N,193,193,2,3.5,0.5,0.5,0,5.76,0.3,13.31,',
 '2,11/28/2018 16:29,11/28/2018 16:33,5,0,2,N,193,193,2,52,0,0.5,0,0,0.3,55.55,',
 '1,1/1/2019 0:21,1/1/2019 0:28,1,1.3,1,N,163,229,1,6.5,0.5,0.5,1.25,0,0.3,9.05,',
 '1,1/1/2019 0:32,1/1/2019 0:45,1,3.7,1,N,229,7,1,13.5,0.5,0.5,3.7,0,0.3,18.5

In [29]:
df_new = rdd.\
    map(lambda x: x.split(",")).\
    filter(lambda x: len(x) == 18). \
    map(lambda x: {
        'VendorID':x[0],
        'tpep_pickup_datetime':x[1],
        'tpep_dropoff_datetime':x[2],
        'passenger_count':x[3],
        'trip_distance':x[4],
        'RatecodeID':x[5],
        'store_and_fwd_flag':x[6],
        'PULocationID':x[7],
        'DOLocationID':x[8],
        'payment_type':x[9],
        'fare_amount':x[10],
        'extra':x[11],
        'mta_tax':x[12],
        'tip_amount':x[13],
        'tolls_amount':x[14],
        'improvement_surcharge':x[15],
        'total_amount':x[16],
        'congestion_surcharge':x[17]})\
    .toDF()
df_new.show(5)

/usr/lib/spark/python/pyspark/sql/session.py:366: UserWarning: Using RDD of dict to inferSchema is deprecated. Use pyspark.sql.Row instead
  warnings.warn("Using RDD of dict to inferSchema is deprecated. "


+------------+------------+----------+--------+--------------------+-----+-----------+---------------------+-------+---------------+------------+------------------+----------+------------+------------+---------------------+--------------------+-------------+
|DOLocationID|PULocationID|RatecodeID|VendorID|congestion_surcharge|extra|fare_amount|improvement_surcharge|mta_tax|passenger_count|payment_type|store_and_fwd_flag|tip_amount|tolls_amount|total_amount|tpep_dropoff_datetime|tpep_pickup_datetime|trip_distance|
+------------+------------+----------+--------+--------------------+-----+-----------+---------------------+-------+---------------+------------+------------------+----------+------------+------------+---------------------+--------------------+-------------+
|DOLocationID|PULocationID|RatecodeID|VendorID|congestion_surcharge|extra|fare_amount| improvement_surch...|mta_tax|passenger_count|payment_type|store_and_fwd_flag|tip_amount|tolls_amount|total_amount| tpep_dropoff_date...|

In [6]:
#df = (sqlContext.read
#         .format("com.databricks.spark.csv")
#         .option("header", "true")
#         .load("s3://msbx5420-2020/team-blanca-peak/yellow_tripdata_2019-01.csv"))

#spark.read.format("csv").option("header", "true").load("s3://msbx5420-2020/team-blanca-peak/yellow_tripdata_2019-01.csv") 
df_read = spark.read.option("header", "true").csv("s3://msbx5420-2020/team-blanca-peak/yellow_tripdata_2019-01.csv")
df_read.show(5)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|       1|       1/1/2019 0:46|        1/1/2019 0:53|              1|          1.5|         1|                 N|         151|         239|           1|          7|  0.5|    0.5|      1.65|           0|                  0.3

In [7]:
#Register the people df as a table
df_read.registerTempTable("taxi")
#Perform SQL Query
taxi = spark.sql("SELECT * FROM taxi")
taxi.show(5)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|       1|       1/1/2019 0:46|        1/1/2019 0:53|              1|          1.5|         1|                 N|         151|         239|           1|          7|  0.5|    0.5|      1.65|           0|                  0.3

In [8]:
df_write = rdd.\
    map(lambda x: x.split(",")).toDF()
df_write.show(5)

+--------+--------------------+--------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+--------------------+------------+--------------------+
|      _1|                  _2|                  _3|             _4|           _5|        _6|                _7|          _8|          _9|         _10|        _11|  _12|    _13|       _14|         _15|                 _16|         _17|                 _18|
+--------+--------------------+--------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+--------------------+------------+--------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_date...|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surch...|total_a

In [ ]:
df_write.write.parquet("s3://msbx5420-2020/team-blanca-peak/parquet")

In [18]:
df_read = sqlContext.read.parquet("s3://msbx5420-2020/team-blanca-peak/parquet")
df_read.show(5)

+--------+--------------------+--------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+--------------------+------------+--------------------+
|      _1|                  _2|                  _3|             _4|           _5|        _6|                _7|          _8|          _9|         _10|        _11|  _12|    _13|       _14|         _15|                 _16|         _17|                 _18|
+--------+--------------------+--------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+--------------------+------------+--------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_date...|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surch...|total_a

In [32]:
print(df_new.count())
print(df_read.count())

1048576
1048576


In [ ]:
sc.stop()